In [27]:
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
import pandas as pd
import os
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

import tensorflow_hub as hub
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split


max_words = 15000
tokenizer = Tokenizer(num_words=max_words)
maxlen = 150




In [64]:
module_url = "https://tfhub.dev/google/elmo/3"
embed = hub.load("/Users/shantanu/elmo_3")

In [29]:
root_dir = r'/Users/shantanu/Library/Mobile Documents/com~apple~CloudDocs/Capstone/Wikipedia Data'

result_path = annot_file_path = os.path.join(root_dir , 'comments_with_grouped_annoptations.tsv')

merged_comments = pd.read_table(result_path)
merged_comments['recipient_attack'] = merged_comments['recipient_attack'].apply(lambda x : 1 if x> 1 else 0 )
X_train = merged_comments['comment']

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)
sequences = tokenizer.texts_to_sequences(X_train)


data = pad_sequences(sequences, maxlen=maxlen)
print('Shape of data tensor:', data.shape)
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
y = merged_comments['recipient_attack']

one_hot_train_labels = to_categorical(y)

labels = np.asarray(one_hot_train_labels)
print('Shape of label tensor:', labels.shape)
labels = labels[indices]


X_train, X_test_validate, y_train, y_test_validate = train_test_split(merged_comments.comment, merged_comments.recipient_attack,
                                                    stratify = labels,
                                                    test_size = 40,
                                                    random_state = 123)

DF_train = X_train.to_frame()
DF_train['recipient_attack'] = y_train

DF_train.reset_index(inplace=True)

Shape of data tensor: (115864, 150)
Shape of label tensor: (115864, 2)


In [30]:
import pickle
with open("../tokenizer.pkl","rb") as f:
    tokenizer = pickle.load(f)

In [31]:
### Model load 

model = load_model('../basic_lstm_with_auto_encoder.h5')


In [46]:
import time
import random
import os


def preprocess_text(text_eval):
    sequences = tokenizer.texts_to_sequences([text_eval])
    data = pad_sequences(sequences, maxlen=maxlen)
    return data


def abalation(args):
    ret = dict()
    index, row = args
    print("running for {}".format(index))
    text = row['comment'].translate(str.maketrans('', '', string.punctuation))
    tokens = word_tokenize(text)
    tokens_without_sw = tokens  # [word for word in tokens if not word in stopwords.words()]
    text = tf.convert_to_tensor([str(tokens_without_sw)])
    out = embed.signatures['default'](text)['elmo']
    pred_vals_array = np.empty((0, 2), float)
    for i in range(len(tokens_without_sw) - 2):
        text_eval = tokens_without_sw[i:i + 3]
        a = preprocess_text(text_eval)
        pred_vlaues = model.predict(a)
        pred_vals_array = np.vstack((pred_vals_array, pred_vlaues))
    elmo_vecs = update_mat(pred_vals_array, out, row.recipient_attack)
    ret[row.recipient_attack] = elmo_vecs
    return ret

def update_mat(pred_vals_array, out, recipient_attack):
    phrase_start_index = pred_vals_array[:, recipient_attack].argmax()
    elmo_vecs_NAE = np.array(out[0][phrase_start_index:phrase_start_index + 3])
    return elmo_vecs_NAE



In [47]:
DF_train.tail()

,index,comment,recipient_attack
574,6382,`NEWLINE_TOKENNEWLINE_TOKEN:Your not the only ...,1
575,97303,.NEWLINE_TOKENNEWLINE_TOKEN== Remove Legends =...,0
576,64439,`::The differences in diagnosis rates is likel...,0
577,37389,NEWLINE_TOKENNEWLINE_TOKEN== Eugen Vögler / Al...,0
578,10229,"NEWLINE_TOKEN:Interesting, hadn't noticed that...",1


In [48]:
def run_sequential ():
    for rec in DF_train.iterrows():
        ret = abalation(rec)
        print(ret)


In [60]:
a = list(map(abalation,DF_train.iloc[0:9999,:].iterrows()))

running for 7
running for 8
running for 9


In [61]:
a

[{0: array([[-0.18450694,  0.0786566 , -0.00160237, ...,  0.09328601,
           0.30909327, -0.7406552 ],
         [-0.28309616,  0.40874362,  0.33764035, ...,  0.22987443,
           0.63209915, -0.41705024],
         [-0.44029447,  0.1612227 ,  0.28173673, ...,  0.00222512,
           0.11175109, -0.46779114]], dtype=float32)},
 {1: array([[-0.23287071,  0.00908137,  0.09652573, ...,  0.11265226,
           0.33093327, -0.287987  ],
         [-0.6177983 , -0.10961051,  0.5007458 , ...,  0.06490551,
           0.70277816,  0.01085995],
         [-0.04294208,  0.17718947,  0.16293955, ...,  0.17081118,
          -0.37689048, -0.578506  ]], dtype=float32)},
 {0: array([[-0.89535284, -0.2372038 ,  0.6088952 , ...,  0.08640732,
           1.0694932 , -0.7675009 ],
         [ 0.05310196, -0.12529181,  0.428167  , ..., -0.0774144 ,
           0.05584092, -0.6399034 ],
         [-0.4278134 ,  0.05618691,  0.29306653, ...,  0.08047161,
           0.61658835, -0.04825191]], dtype=float32)}]

In [62]:
pd.DataFrame.from_records(a)

,0,1
0,"[[-0.18450694, 0.0786566, -0.0016023666, 0.100...",NaN
1,NaN,"[[-0.23287071, 0.009081371, 0.09652573, 0.2598..."
2,"[[-0.89535284, -0.2372038, 0.6088952, 0.818773...",NaN


In [59]:
a[0]

{0: array([[-0.3210175 ,  0.17366603,  0.5446634 , ...,  0.34253007,
          0.75947845, -0.21353534],
        [-0.49913216, -0.04212511, -0.05902478, ...,  0.63415277,
          0.55470955, -0.35010707],
        [-0.10557193, -0.10751615,  0.4985875 , ..., -0.08313151,
          0.7551563 ,  0.00652687]], dtype=float32)}

In [49]:
run_sequential()

running for 0
{0: array([[-0.3210175 ,  0.17366603,  0.5446634 , ...,  0.34253007,
         0.75947845, -0.21353534],
       [-0.49913216, -0.04212511, -0.05902478, ...,  0.63415277,
         0.55470955, -0.35010707],
       [-0.10557193, -0.10751615,  0.4985875 , ..., -0.08313151,
         0.7551563 ,  0.00652687]], dtype=float32)}
running for 1
{0: array([[-0.41099688, -0.02160566,  0.04283985, ..., -0.13623561,
         0.907256  , -0.12000874],
       [-0.2678803 , -0.04774128,  0.4615938 , ...,  0.14850183,
         1.5514276 , -0.02096853],
       [-0.20045634,  0.19181986, -0.219108  , ..., -0.21944699,
         0.6316541 , -0.43269682]], dtype=float32)}
running for 2
{0: array([[-0.22160067, -0.26873082,  0.38253418, ..., -0.02454223,
         0.4395522 , -0.3907471 ],
       [-0.5252863 ,  0.01354215,  0.5582873 , ...,  0.73088264,
         0.3109154 , -0.47041026],
       [-0.08268154, -0.27157763,  0.15044408, ...,  0.1597502 ,
         0.7503923 , -0.54363966]], dtype=float

KeyboardInterrupt: 

In [43]:
ret

NameError: name 'ret' is not defined